### IMPORTS:

In [ ]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import re
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from plotnine import *

### Conexión a MongoDB:

In [ ]:
cliente = MongoClient('localhost', 27017)
base_de_datos = cliente['scrap_link']
collection = base_de_datos['scrap_perfiles']                     # colección donde guardamos los datos
collection.create_index([("enlace", pymongo.DESCENDING)]) 

## EDUCACIÓN:

#### Query_1: Institucion = Ironhack;               cols. = nombre, institucion, fecha_inicio, fecha_fin

In [ ]:
# Query mediante Mongo db:

query_1 =  {
    
    "educación.educacion": {                                   #accedemos a las tarjetas educación, a la key educacion(lista)
        "$elemMatch":{                                         # elemento que coincida con: Ironhack
            "institucion" : "Ironhack"
            
        }       
    }
}

projection = {                                                  #datos que cojemos:
                                                                #_id que empiece en 0, nombre sí, query sí
    "nombre" : 1,
    "educación.educacion.$": 1
    
}


result_1 = collection.find (query_1, projection)

data = []

for r in result_1:
    
    nombre = r.get("nombre")
    ironhack = r.get("educación", {}).get("educacion", [{}])
    for e in ironhack:
        e["nombre"] = nombre
        data.append(e)
df_educacion = pd.DataFrame(data)

In [ ]:
df_educacion.head()

In [ ]:
df_educacion.info()

### Trabajamos con los nulos:

In [ ]:
# Rellenamos los datos nulos en una col. con los datos de la otra y viceversa:

def date_null(df, cols):
    
    df[cols[0]].fillna(df[cols[1]], inplace=True)
    df[cols[1]].fillna(df[cols[0]], inplace=True)

date_null(df_educacion, ['fecha_inicio', 'fecha_fin'])

In [ ]:
#Resto de nulos:

filas_nuls = df_educacion.isnull().any(axis = 1)
df_educacion[filas_nuls]

In [ ]:
# Nos quedan dos datos, los ventilamos porque no sirven para el anailis:
df_educacion = df_educacion.dropna()
df_educacion.info()

### Trabajamos con la columna carrera:

In [ ]:
# Convertimos todo el texto a minus:
df_educacion.carrera = df_educacion.carrera.str.lower()

In [ ]:
# Creamos una funcion para crear una nueva columna: bootcamp, para saber que bootcamp han hecho:

def bootcamp(value):
    if "web" in value or "front" in value or "stack" in value or "development" in value:
        return "desarrollo web"
    elif "data" in value or "dato" in value:
        return "análisis de datos"
    elif "ux/ui" in value or "ux" in value or "ui" in value:
        return "diseño UX/UI"
    elif "ciber" in value or "cyber" in value:
        return "ciberseguridad"
    else:
        return "otro"
    
        return value

In [ ]:
df_educacion["bootcamp"] = df_educacion.carrera.apply(bootcamp)
df_educacion.head()

### Trabajamos con las columnas de fecha:

In [ ]:
# Funciones para limpiar las fechas:

def clean_date(fecha_str):
    fecha_str.strip()
    partes = fecha_str.split()
    if len(partes) == 2:
        mes = partes[0]
        año = partes[1]
        mes_num = {
            'ene.': '01', 'feb.': '02', 'mar.': '03', 'abr.': '04',
            'may.': '05', 'jun.': '06', 'jul.': '07', 'ago.': '08',
            'sept.': '09', 'oct.': '10', 'nov.': '11', 'dic.': '12'
        }
        fecha_nueva = f"{año}-{mes_num[mes]}"
        return (fecha_nueva)                                                            # Convertir a objeto datetime
    else:
        return fecha_str                                                                # Devolver solo el año como número entero
    
    
def date_to_datetime(fecha):
    
    try:                                                                                # Intenta convertir a formato 'año-mes
        fecha_convertida = pd.to_datetime(fecha, format='%Y-%m')#.date()     
    except ValueError:
        
        try:                                                                            # Intenta convertir a formato 'año'                                                                                                                            # Intenta convertir a formato 'año'                                                                                        # Intenta convertir a formato 'año'
            fecha_convertida = pd.to_datetime(fecha, format='%Y')#.date()
            
        except ValueError:
                                                                                        # Si no coincide con ninguno de los formatos, mantener como NaN
            fecha_convertida = None
    
    return fecha_convertida

In [ ]:
# Aplicar la función a las columnas 'fecha_inicio' y 'fecha_fin'

df_educacion['fecha_inicio'] = df_educacion['fecha_inicio'].apply(clean_date)
df_educacion['fecha_fin'] = df_educacion['fecha_fin'].apply(clean_date)

In [ ]:
df_educacion['fecha_inicio'] = df_educacion['fecha_inicio'].apply(date_to_datetime)
df_educacion['fecha_fin'] = df_educacion['fecha_fin'].apply(date_to_datetime)
df_educacion.head()

In [ ]:
df_educacion.info()

### Ordenamos las columnas y quitamos la de institución , la de carrera y la de fecha de inicio:

In [ ]:
orden_col = ["nombre", "bootcamp", "fecha_fin"]
df_educacion.drop(columns = ["institucion", "carrera"], axis = 1, inplace = True)
df_educacion = df_educacion[orden_col]

In [ ]:
df_educacion.head()

## EXPERIENCIA

#### Query_2: experiencias_laborales

In [ ]:
# Query mediante python:

def extract_data(document):
    nombre = document.get("nombre", "")
    experiencias = document.get("experiencia_laboral", {}).get("experiencias", [])
    data = []
    for experiencia in experiencias:
        nombre_empresa = experiencia.get("nombre_empresa", "")
        puestos = experiencia.get("puestos", [])
        for puesto in puestos:
            fecha_inicio = puesto.get("fecha_inicio", "")
            ubicacion = puesto.get("ubicacion", "")
            nombre_puesto = puesto.get("nombre_puesto", "")
            data.append([nombre, nombre_empresa, fecha_inicio, ubicacion, nombre_puesto])
    return data

documents = collection.find({})                                                             # Obtener todos los documentos de la colección
data = []

for document in documents:
    data.extend(extract_data(document))
    

df_trabajos = pd.DataFrame(data, columns=["nombre", "nombre_empresa", "fecha_inicio", "ubicacion", "nombre_puesto"])

In [ ]:
df_trabajos.head()

In [ ]:
df_trabajos.info()                                        #No hay nulos

### Trabajamos con las columnas de fecha:

In [ ]:
# Algunas fechas tienen texto que no queremos:

df_trabajos.fecha_inicio = df_trabajos.fecha_inicio.str.split(' · ').str[0]            #dividimos los valores string por el punto y nos quedamos la primera parte
df_trabajos   

In [ ]:
# Aplicar la funciones definidas antes a la columna 'fecha_inicio':

df_trabajos['fecha_inicio'] = df_trabajos['fecha_inicio'].apply(clean_date)
df_trabajos['fecha_inicio'] = df_trabajos['fecha_inicio'].apply(date_to_datetime)

### Trabajamos con la columna nombres_empresa:

In [ ]:
# Limpiamos los nombres:
df_trabajos.nombre_empresa.unique().shape

In [ ]:
# Eliminamos lo que no sea el nombre de la empresa y convertimos a minusculas:

df_trabajos.nombre_empresa = df_trabajos.nombre_empresa.str.split(' · ').str[0].str.lower()
df_trabajos.nombre_empresa.unique().shape

In [ ]:
# vamos a hacer algunos ajustes:
palabras_clave = ['vodafone', 'santander', 'accenture', 'openbank', 'ironhack', 'deloitte', 'bbva','universidad rey juan carlos', 'amazon', 'inditex', 'ntt', 'ibm', 'kpmg' ]

for palabra in palabras_clave:
    df_trabajos['nombre_empresa'] = df_trabajos['nombre_empresa'].str.replace(rf'.*{palabra}.*',palabra.capitalize(), case=False, regex=True)

df_trabajos.nombre_empresa.unique().shape    

In [ ]:
# Comprobamos si hay algo más que limpiar, en las dos siguientes celdas:
df_trabajos.nombre_empresa.unique()

In [ ]:
# Usado para limpiar los datos unicos de experiencias:
#filas_busqueda = df_experiencias[df_experiencias['nombre_empresa'].str.contains('rellenar_empresa', case=False)]
#filas_busqueda

In [ ]:
# Cambiamos el nombre de la col nombre_empresa por empresa:
df_trabajos = df_trabajos.rename(columns={'nombre_empresa': 'empresa'})

In [ ]:
df_trabajos.info()

In [ ]:
# agrupamos freelance, autonomo y profesional independiente bajo el mismo nombre: freelance.
df_trabajos['empresa'] = df_trabajos['empresa'].str.lower()

df_trabajos['empresa'].replace(['autonomo', 'profesional independiente'], 'freelance', inplace=True)
df_trabajo

### Comprobamos que la persona ha estudiado en Ironhack y ha conseguido trabajo:

In [ ]:
# Sacamos los valores unicos de la columna nombre de df_educacion:
nombres_educacion = df_educacion['nombre'].unique()


In [ ]:
# Limpiamos el df_experiencias, para eliminar los datos de las personas que no hayan estudiado en Ironhack:
df_experiencias = df_trabajos[df_trabajos.nombre.isin(nombres_educacion)]
df_experiencias.shape

In [ ]:
df_experiencias.info()

In [ ]:
df_experiencias.head()

## Cruzamos los dataframes:

In [ ]:
# Vamos a ffiltrar los dos dataframes
dataframes_filtrados = []

for nombre, grupo in df_experiencias.groupby('nombre'):
    fecha_maxima_fin_educacion = df_educacion[df_educacion['nombre'] == nombre]['fecha_fin'].max()
    grupo_filtrado = grupo[grupo['fecha_inicio'] >= fecha_maxima_fin_educacion]
    dataframes_filtrados.append(grupo_filtrado)

# Concatenar los DataFrames filtrados
df_experiencias_despues_ironhack = pd.concat(dataframes_filtrados)

In [ ]:
df_experiencias_despues_ironhack.head()

In [ ]:
#queremos añadir la columna bootcamp en este dataframe:
df_experiencias_despues_ironhack = df_experiencias_despues_ironhack.merge(df_educacion[['nombre', 'bootcamp']], 
                                                                          on='nombre', how='left')
df_experiencias_despues_ironhack.head()

In [ ]:
# ordenamos el dataframe por los valores de nombre(aunque no sea necesario) y los de fecha de inicio
df_experiencias_despues_ironhack = df_experiencias_despues_ironhack.sort_values(['nombre', 'fecha_inicio']).reset_index()

# Mantener solo las primeras filas para cada nombre (las que tienen las fechas más pequeñas)
df_experiencias_filtrado = df_experiencias_despues_ironhack.groupby('nombre').first().reset_index()

In [ ]:
df_experiencias_filtrado.shape

In [ ]:
# Limpiamos el df_educacion para solo tener personas que hayan encontrado trabajo

nombres_experiencias_filtrado = df_experiencias_filtrado['nombre'].unique()

df_educacion_con_trabajo = df_educacion[df_educacion.nombre.isin(nombres_experiencias_filtrado)]
df_educacion_con_trabajo.shape

In [ ]:
df_educacion_con_trabajo.head()

In [ ]:
df_educacion_con_trabajo = df_educacion_con_trabajo.drop_duplicates()

In [ ]:
# Creamos un dataframe con las columnas combinadas de df_educacion y df_experiencias_filtrado:
df_final = df_experiencias_filtrado.merge(df_educacion_con_trabajo, on='nombre', how='inner')

In [ ]:
df_final.head()

In [ ]:
df_final.shape

In [ ]:
# Tenemos una columna duplicada:

df_final = df_final.drop(['bootcamp_y'], axis =1)

df_final = df_final.rename(columns={'bootcamp_x': 'bootcamp'})
df_final

In [ ]:
# Ordenamos las columnas:
orden_col = ["nombre", "bootcamp", "fecha_fin", "empresa", "nombre_puesto", "fecha_inicio"]
df_final = df_final[orden_col]
df_final

### Dataframe final y EDA:

### Limpiamos la columna de nombre_empresa:

In [ ]:
df_final.empresa.unique()

In [ ]:
df_final.empresa.unique().shape

In [ ]:
#importante para ver agrupados los datos de bootcamp con el puesto para poder hacer una categorización después
df_final[['bootcamp', 'nombre_puesto']].value_counts().nlargest(20)    

In [ ]:
# Creamos una categorización a lo bruto. Next steps: categorización por cercanía de palabras:

dict_postitions_to_category = {}

for position in df_final['nombre_puesto'].unique():
    if ('frontend' in position.lower()) or ('full stack' in position.lower()) or ('backend' in position.lower()) or ('front-end' in position.lower()) or ('full-stack' in position.lower()) or ('front end' in position.lower()) or ('developer' in position.lower()) or ('programador' in position.lower()) or ('desarrollador' in position.lower()):
        dict_postitions_to_category[position] = 'Web Developer'
    elif ('ux' in position.lower()) or ('design' in position.lower()) or ('diseñadora' in position.lower()) or ('ui' in position.lower()):
        dict_postitions_to_category[position] = 'UX/UI design'
    elif('data' in position.lower()) or ('datos' in position.lower() or 'analyst' in position.lower()):
        dict_postitions_to_category[position] = 'Data'
    else:
        dict_postitions_to_category[position] = 'Other'

# Creamos una nueva columna con la categoría creada arriba:
df_final['categoria_puesto'] = (
    df_final['nombre_puesto'].map(dict_postitions_to_category)
)


df_final[['bootcamp', 'categoria_puesto']].value_counts().nlargest(20)    #importante sergio


In [ ]:
# Para poder ver el bootcamp y el puesto:

show_me = df_final.loc[
    (df_final['bootcamp'] == 'análisis de datos')       # ir cambiando bootcamp y puesto para ver datos
    & (df_final['categoria_puesto'] == 'Other')
]
show_me.head(25)

In [ ]:
#Análisis del tiempo que se tarda en conseguir trabajo:

def compute_month_fin(x):
    return pd.to_datetime(x['fecha_fin']).dt.month
def compute_year_fin(x):
    return pd.to_datetime(x['fecha_fin']).dt.year
def compute_month(x):
    return pd.to_datetime(x['fecha_inicio']).dt.month
def compute_year(x):
    return pd.to_datetime(x['fecha_inicio']).dt.year

(
        df_final
        .assign(
            mes_final=compute_month_fin,
            año_final=compute_year_fin,
            mes_inicio=compute_month,
            año_inicio=compute_year
            
            
        )
)



In [ ]:
# Añadimos una columan al dataframe de tiempo en conseguir primer trabajo:
df_final['diferencia_meses'] = (df_final['fecha_inicio'] - df_final['fecha_fin']) 
df_final

In [ ]:
# Eliminamos las filas cuyos valores de fecha sea enero para eliminar el seesgo linkedin:
df_final_january_off = df_final[~((df_final['fecha_fin'].dt.month == 1)|(df_final['fecha_inicio'].dt.month == 1))]
df_final_january_off

In [ ]:
def asignar_tiempo_trabajo(diferencia_timedelta):
    # Extraer el número de días de la diferencia Timedelta
    diferencia_dias = diferencia_timedelta.days
    
    if diferencia_dias <= 90:
        return "Menos de 3 meses"
    elif 91 <= diferencia_dias <= 180:
        return "3-6 meses"
    elif 181 <= diferencia_dias < 365:
        return "6 meses - 1 año"
    else:
        return "Más de 1 año"

# Aplicar la función a la columna "diferencia_meses" para crear la columna "tiempo_trabajo"
df_final_january_off['tiempo_trabajo'] = df_final_january_off['diferencia_meses'].apply(asignar_tiempo_trabajo)

# Crear el gráfico de barras
plt.figure(figsize=(8, 6))
conteo_tiempo = df_final_january_off['tiempo_trabajo'].value_counts()
conteo_tiempo.plot(kind='bar', color='skyblue')
plt.title('Distribución de Tiempo de Trabajo')
plt.xlabel('Tiempo de Trabajo')
plt.ylabel('Número de Personas')
plt.xticks(rotation=45)  # Rotar etiquetas en el eje x para mayor legibilidad
plt.tight_layout()
plt.show()

In [ ]:
porcentajes = df_final_january_off['tiempo_trabajo'].value_counts(normalize=True) * 100

# Crear el gráfico de barras con porcentajes
plt.figure(figsize=(8, 6))
porcentajes.plot(kind='bar', color='skyblue')
plt.title('Distribución de Tiempo de Trabajo en Porcentaje')
plt.xlabel('Tiempo de Trabajo')
plt.ylabel('Porcentaje')
plt.xticks(rotation=45)

# Agregar etiquetas de porcentaje en las barras
for i, porcentaje in enumerate(porcentajes):
    plt.text(i, porcentaje, f'{porcentaje:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
df_final_january_off.head()

In [ ]:
filas_nuls = df_final_january_off.isnull().any(axis = 1)
df_final_january_off[filas_nuls]

In [ ]:
df_to_plot = (
    (
        df_final
        .assign(mes=lambda x: pd.to_datetime(df_final['fecha_fin']).dt.month)
        .groupby(['bootcamp', 'mes', 'categoria_puesto'])
        .size()
        / df_final.assign(mes=lambda x: pd.to_datetime(df_final['fecha_fin']).dt.month).groupby(['bootcamp', 'mes']).size()
    )
    .reset_index(name='pct_users')
    .assign(pct_users=lambda x: round(x['pct_users'] * 100, 2))
)

df_to_plot.sample(5)

In [ ]:
new_plot = list(
    it.product(df_to_plot['bootcamp'].unique(),
               df_to_plot['mes'].unique(),
               df_to_plot['categoria_puesto'].unique())
)

df_new_plot = pd.DataFrame(
    new_plot, columns=['bootcamp', 'mes', 'categoria_puesto']
)

df_to_plot_filled = (
    df_new_plot
    .merge(df_to_plot, on=['categoria_puesto', 'mes', 'bootcamp'], how='left')
    .fillna(0)
)

df_plot_puestos = df_to_plot_filled.loc[lambda x: x['pct_users'] > 0]
df_plot_puestos

In [ ]:
show_me = df_to_plot.loc[df_to_plot.bootcamp == 'análisis de datos']
show_me

In [ ]:
bootcamp_with_interest = [
    'análisis de datos', 'desarrollo web', 'diseño UX/UI'
]

(
    ggplot(df_to_plot_filled[
        df_to_plot_filled['bootcamp'].isin(bootcamp_with_interest)
    ], aes(x='mes', y='pct_users', color='categoria_puesto', label='pct_users'))
    + facet_wrap('bootcamp', ncol=1)
    + geom_line()
    + geom_text()
    + scale_x_discrete(limits=range(1, 12 + 1))
    + theme(figure_size=(14, 8))
)

In [ ]:
from plotnine import *


df_to_plot = (
    (
        df_final
        .assign(mes=lambda x: pd.to_datetime(df_final['fecha_fin']).dt.month)
        .groupby(['bootcamp', 'mes', 'categoria_puesto'])
        .size()
    )
    .reset_index(name='num_users')
)

df_to_plot.sample(5)

In [ ]:
new_plot = list(
    it.product(df_to_plot['bootcamp'].unique(),
               df_to_plot['mes'].unique(),
               df_to_plot['categoria_puesto'].unique())
)

df_new_plot = pd.DataFrame(
    new_plot, columns=['bootcamp', 'mes', 'categoria_puesto']
)

df_to_plot_filled = (
    df_new_plot
    .merge(df_to_plot, on=['categoria_puesto', 'mes', 'bootcamp'], how='left')
    .fillna(0)
)

df_to_plot_filled.loc[lambda x: x['num_users'] > 0]

In [ ]:
bootcamp_with_interest = [
    'análisis de datos', 'desarrollo web', 'diseño UX/UI'
]

(
    ggplot(df_to_plot_filled[
        df_to_plot_filled['bootcamp'].isin(bootcamp_with_interest)
    ], aes(x='mes', y='num_users', color='categoria_puesto', fill='categoria_puesto', label='num_users'))
    + facet_wrap('bootcamp', ncol=1)
    + geom_col(position='dodge2')
    + geom_text()
    + scale_x_discrete(limits=range(1, 12 + 1))
    + theme(figure_size=(16, 10))
)

In [ ]:
(
        df_final
        .groupby(['bootcamp', 'categoria_puesto'])
        .size()
    ).index

In [ ]:
df_final.groupby('bootcamp').size().index

In [ ]:
df_pct_users_by_bootcamp_and_category_position = (
    (
        df_final
        .groupby(['bootcamp', 'categoria_puesto'])
        .size()
        / df_final.groupby('bootcamp').size()
    )
    .reset_index(name='pct_users')
)
    
(
    df_pct_users_by_bootcamp_and_category_position
    .style
    .format('{:.2%}', subset='pct_users')
)

#### Analizamos los teachers assistant de Ironhack:

In [ ]:
filas_busqueda = df_final[df_final['empresa'].str.contains('ironhack', case=False)]
filas_busqueda.nombre_puesto = filas_busqueda.nombre_puesto.astype(str)

In [ ]:
valor_busqueda = 'assi'
reemplazo = 'Teacher Assistant'

# Aplicar la sustitución a la columna 'columna'
filas_busqueda['nombre_puesto'] = filas_busqueda['nombre_puesto'].str.replace('.*' + valor_busqueda + '.*', 
                                                                              reemplazo, regex=True)

In [ ]:
filas_busqueda['nombre_puesto'] = filas_busqueda['nombre_puesto'].str.lower()
filas_busqueda

In [ ]:

palabras_clave = ['assistant', 'asistente','ayudante']

# Función que reemplazará los valores
def reemplazar_si_contiene(valor):
    for palabra in palabras_clave:
        if palabra in valor:
            return 'teacher assistant'
    return valor

# Aplicar la función a la columna 'nombre_puesto'
filas_busqueda['nombre_puesto'] = filas_busqueda['nombre_puesto'].apply(reemplazar_si_contiene)

In [ ]:
ilas_busqueda.loc[filas_busqueda['nombre_puesto']== 'teacher assistant'].shape

### Convert to csv:

### df_educacion:

In [ ]:
df_educacion = df_educacion.drop(['nombre'], axis=1)
df_educacion.to_csv("df_educacion.csv", index=False)

In [ ]:
# quitamos los datos de enero
df_educacion_no_january = df_educacion[df_educacion['fecha_fin'].dt.month != 1]
df_educacion_no_january.head()

In [ ]:
df_educacion_no_january.to_csv("df_educacion_no_january.csv", index=False)

### df_experiencias: 

In [ ]:
df_experiencias = df_experiencias.drop(['nombre'], axis =1)
df_experiencias.head()

In [ ]:
df_experiencias.to_csv("df_experiencias.csv", index = False)

### df_final

In [ ]:
df_final.to_csv("df_final.csv", index = False)

### df_experiencias_despues_ironhack

In [ ]:
df_experiencias_despues_ironhack = df_experiencias_despues_ironhack.drop(['nombre'], axis =1)
df_experiencias_despues_ironhack.to_csv("df_experiencias_despues_ironhack.csv", index = False)

### df_experiencias_filtrado

In [ ]:
df_experiencias_filtrado = df_experiencias_filtrado.drop(['nombre'], axis = 1)
df_experiencias_filtrado = df_experiencias_filtrado.drop(['index'], axis = 1)
df_experiencias_filtrado.head()

In [ ]:
df_experiencias_filtrado.to_csv("df_experiencias_filtrado.csv", index = False)